In [60]:
from fastai import * 
from fastai.text import * 
import numpy as np

In [103]:
            import tracemalloc
            tracemalloc.start()
            snapshot1 = tracemalloc.take_snapshot()

            -----your interesting code
            
            snapshot2 = tracemalloc.take_snapshot()
            top_stats = snapshot2.compare_to(snapshot1, 'lineno')
            print(f"Top 10 of {len(top_stats)}")
            for stat in top_stats[:10]: print(stat)
          

(28,)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27]


In [106]:
data = buffer[:bs*3].reshape(bs,3)

In [123]:
-1%9

8

In [122]:
l=9
for i in range(2*l):print(f"i:{i} i-backwards:{l-1 - i%l}")

i:0 i-backwards:8
i:1 i-backwards:7
i:2 i-backwards:6
i:3 i-backwards:5
i:4 i-backwards:4
i:5 i-backwards:3
i:6 i-backwards:2
i:7 i-backwards:1
i:8 i-backwards:0
i:9 i-backwards:8
i:10 i-backwards:7
i:11 i-backwards:6
i:12 i-backwards:5
i:13 i-backwards:4
i:14 i-backwards:3
i:15 i-backwards:2
i:16 i-backwards:1
i:17 i-backwards:0


In [87]:
seq_len=3
a = np.arange(bs*seq_len,-1,-1)
ix = np.arange(len(a))
print(ix)
print(a)
ts[ix] = a[ix]
print(t.shape)
print(t)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
[12 11 10  9  8  7  6  5  4  3  2  1  0]
torch.Size([28])
tensor([12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27])


In [88]:
t.numel()/3

9.333333333333334

In [89]:
bs=4
sl=3
bs_dim=t.numel()/sl
t.view(-1,14)

tensor([[12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0, 13],
        [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]])

In [100]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


In [ ]:
import asyncio
async def myfun():
    await asyncio.sleep(1)
    return 5

In [ ]:
await myfun()
await myfun()


In [ ]:
async def hello():
    await asyncio.sleep(1)
    return "hello"
async def test():
    loop = asyncio.get_event_loop()
    tasks = []
    for i in range(10):
        tasks.append(loop.create_task(hello()))
    # all the tasks will automatically run
    asyncio.set_event_loop(loop)
    rest = asyncio.gather(*tasks)
    await rest

In [ ]:
#await test()

loop = asyncio.get_event_loop()
tasks = []
for i in range(10):
    tasks.append(loop.create_task(hello()))
# all the tasks will automatically run
asyncio.set_event_loop(loop)
rest = asyncio.gather(*tasks)
await rest

In [ ]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool 

pool = ThreadPool(1) 
results = pool.map(test,None)
pool.close() 
pool.join()


In [ ]:
        """
        async def hello():
            await asyncio.sleep(1)
            return "hello"
    
        fileblocks = partition_by_cores(files, n_cpus)
        print(f"fileblocks:{len(fileblocks)}")
        tasks = []
        i=0
        loop = asyncio.get_event_loop()
        for fb in fileblocks:
            print(f"task:{i}")
            i += 1
            results = tasks.append( loop.create_task( FileTokenizer.getIds_from_file(fb) ) )
        asyncio.set_event_loop(loop)
        results = asyncio.gather(*tasks)
        await results
        """


In [ ]:
lang            = "en"
pathData       = Path("../nlp-data")
path           = pathData / lang
pathTrainValid = path/"wiki-train_valid"
pathNPY        = pathTrainValid/"dummy.npy"
pathNPZ        = pathTrainValid/"dummy.npz"


In [ ]:
arrays = np.empty(10, dtype=object)
arrays

In [ ]:
import io
a=np.arange(3,dtype=np.int32).astype(np.int32)
b=np.arange(5,dtype=np.int32).astype(np.int32)
c=np.arange(10,dtype=np.int32).astype(np.int32)


In [ ]:
with pathNPY.open("wb") as f:
    np.save(f,a, allow_pickle=False, fix_imports=False)
    np.save(f,b, allow_pickle=False, fix_imports=False)
    np.save(f,c, allow_pickle=False, fix_imports=False)

with pathNPY.open("rb") as f:
    f.seek(0, 2); file_size = f.tell(); f.seek(0)
    while f.tell() != file_size:
        print(np.load(f))


In [ ]:
arrays = np.empty(0, dtype=object)
arrays

In [ ]:
arrays = [] #np.empty(3, dtype=object)
for i in range(3):
    #for i,a in enumerate(arrays):
    arrays.append( np.arange(i+1) )

#arrays2 = np.empty(3, dtype=object)
#arrays2[:3] = arrays[:3]
#arrays[1][0] = 10
print(arrays)

def save(arrays):
    with pathNPY.open("wb") as f:
        if isinstance(arrays,list): 
            arrays = np.asarray(arrays,dtype=object)
        np.save(f,arrays, allow_pickle=True, fix_imports=False)
        
def load():
    with pathNPY.open("rb") as f:
        arrays = np.load(f)
    return arrays

save(arrays)
load()

In [ ]:
%timeit save(arrays)

In [ ]:
%timeit load()
print(load())

In [ ]:
arrays = [] #np.empty(0, dtype=object)
for i in range(3):
    a = load()
    if len(a) > 0: arrays.extend( a.tolist() )
arrays = np.asarray(arrays,dtype=object)    
print(len(arrays))
print(arrays)